In [15]:
import sys
import pybullet as p
import pybullet_data
import time
import random
from copy import deepcopy
import numpy as np
path="/its/home/drs25/Documents/GitHub/Sim-Biped-Walker/URDF_files/walker_assembly/"
if sys.platform.startswith('win'):
    path="C:/Users/dexte/Documents/GitHub/Sim-Biped-Walker/URDF_files/walker_assembly/"
robot_urdf_path = "urdf/walker_assembly.urdf"  # Replace with the actual path to your URDF file

sys.path.append("/its/home/drs25/Documents/GitHub/Sim-Biped-Walker/Code/")
import Biped_controller as bp

## Physics and robot creation

In [16]:
def createEngine():
    # Initialize PyBullet
    physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
    # Remove all objects
    num_objects = p.getNumBodies()
    for i in range(num_objects):
        p.removeBody(p.getBodyUniqueId(i))

    # Reset the environment
    p.resetSimulation()
    p.setAdditionalSearchPath(path)  # Set the path for urdf files
    p.setAdditionalSearchPath(pybullet_data.getDataPath())  # Set the path for urdf files
    # Load ground plane
    p.loadURDF("plane.urdf",[0, 0, 0])  # Assuming there's a plane.urdf file in the pybullet data path
    # Load URDF file
    gravity = [0, 0, -9.8]
    p.setGravity(gravity[0], gravity[1], gravity[2])

def reset():
    num_objects = p.getNumBodies()
    for i in range(num_objects):
        p.removeBody(p.getBodyUniqueId(i))
    # Reset the environment
    p.resetSimulation()
    p.loadURDF("plane.urdf",[0, 0, 0])  # Assuming there's a plane.urdf file in the pybullet data path
    # Load URDF file
    gravity = [0, 0, -9.8]
    p.setGravity(gravity[0], gravity[1], gravity[2])

createEngine()
robot=bp.Biped(p,path+robot_urdf_path)

error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.

In [ ]:
print(robot.get_orientation())
start_pos=robot.get_position()
robot.set_positions([0.1,0.5,0.5,0.8])
for _ in range(100):  # Run the simulation for 1000 steps (adjust as needed)
    p.stepSimulation()
    time.sleep(1./240.)
print(robot.orientation)

(-7.403932178333501e-18, -7.822274398480377e-17, 2.2034421452109355e-18)
[-7.403932178333501e-18, -7.822274398480377e-17, 2.2034421452109355e-18]


## Evolution functions

In [ ]:
def fitness_func(robot,startpos,endpos):
    fitness=0
    distances=np.linalg.norm(np.array(startpos) - np.array(endpos))
    orientation=robot.get_orientation()
    dist=np.linalg.norm(np.array(robot.orientation) - np.array(orientation))
    if dist>2:
        #fallen over robot
        return fitness
    return distances

import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights = []
        self.biases = []
        
        # Initialize weights and biases for hidden layers
        for i, layer_size in enumerate(hidden_layers):
            if i == 0:
                input_layer_size = input_size
            else:
                input_layer_size = hidden_layers[i-1]
            self.weights.append(np.random.randn(input_layer_size,layer_size))
            self.biases.append(np.random.randn(1,layer_size))
        
        # Initialize weights and biases for output layer
        self.weights.append(np.random.randn(hidden_layers[-1],output_size))
        self.biases.append(np.random.randn(1,output_size))
        
    def forward(self, x):
        # Forward pass through the network
        for i in range(len(self.weights)):
            z = np.dot(x,self.weights[i]) + self.biases[i]
            x = self.sigmoid(z) if i < len(self.weights) - 1 else self.softmax(z)
        return x
    def count_parameters(self):
        # Count total number of values in weights and biases
        total_parameters = sum(np.size(w) for w in self.weights) + sum(np.size(b) for b in self.biases)
        return total_parameters
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def softmax(self, z):
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=0)
    
    def mutate(self, mutation_array):
        # Mutate weights and biases using Gaussian noise
        index = 0
        for i in range(len(self.weights)):
            self.weights[i] += mutation_array[index: index + self.weights[i].size].reshape(self.weights[i].shape)
            index += self.weights[i].size
        for i in range(len(self.biases)):
            self.biases[i] += mutation_array[index: index + self.biases[i].size].reshape(self.biases[i].shape)
            index += self.biases[i].size
    def set_genes(self, genes):
        # Assign weights and biases using the provided array
        index = 0
        for i in range(len(self.weights)):
            self.weights[i] = genes[index: index + self.weights[i].size].reshape(self.weights[i].shape)
            index += self.weights[i].size
        for i in range(len(self.biases)):
            self.biases[i] = genes[index: index + self.biases[i].size].reshape(self.biases[i].shape)
            index += self.biases[i].size


In [ ]:
nn=NeuralNetwork(5,[10],2)
nn.forward(np.random.normal(0,1,(4,5)))
noise=np.random.normal(0,5,(nn.count_parameters(),))
nn.mutate(noise)
reset()
robot=bp.Biped(p,path+robot_urdf_path,position=[0, 0, .15])
for _ in range(50):  # Run the simulation for 1000 steps (adjust as needed)
    p.stepSimulation()
    time.sleep(1./240.)

## GA

In [17]:
def runTrial(agent,time_max,co=0):
    dt=0.01
    timer=0
    reset()
    robot=bp.Biped(p,path+robot_urdf_path,position=[0, 0, .15])
    start_pos=robot.get_position()
    while timer<time_max:
        friction=np.array([co])
        positions=np.array(robot.positions())
        out=agent.forward(np.concatenate([positions,friction],axis=0))
        robot.set_positions(out.flatten())
        p.stepSimulation()
        #time.sleep(1./240.)
        timer+=dt
    return fitness_func(robot,start_pos,robot.get_position())
def microbial(populalation,generations,mutation_rate,trials=5,co=0):
    std=10*mutation_rate
    fitness=[0]
    for i in range(generations):
        #select genotypes
        idx1=random.randint(0,len(populalation)-1)
        idx2=random.randint(0,len(populalation)-1)
        while idx1==idx2: #prevent same index
            idx2=random.randint(0,len(populalation)-1)
        #run trial
        fit1=0
        for i in range(trials):
            agent=NeuralNetwork(5,[10,20],4)
            agent.set_genes(populalation[idx1])
            fit1+=runTrial(agent,10,co=co)
        fit1/=trials
        fit2=0
        for i in range(trials):
            agent=NeuralNetwork(5,[10,20],4)
            agent.set_genes(populalation[idx2])
            fit2+=runTrial(agent,10,co=co)
        fit2/=trials
        if fit1>fit2: #make tournanment and selection
            populalation[idx2]=deepcopy(populalation[idx1])+np.random.normal(0,std,populalation[idx2].shape)
        else:
            populalation[idx1]=deepcopy(populalation[idx2])+np.random.normal(0,std,populalation[idx1].shape)
        fitness.append(max(fitness+[fit1,fit2]))
    return fitness

population_size=100
num_genes=5*10 + 10*20 + 20*4 +10 + 20 + 4
population=np.random.normal(5,10,(population_size,num_genes))
fitnesses=microbial(population,1000,0.2)
print(fitnesses)